# Kellis Alzheimer paper - pySCENIC pipeline (Embedded version)

**Author:** Vincent Gardeux

**Date Created:** 16/04/2025

**Date Modified:** 16/04/2025

# Libraries

In [1]:
!pip install anndata
# I need anndata to load the h5ad file. Installing within the Docker...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 26.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.11.1 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.

[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Fix OPENBLAS Warnings
import os
default_n_threads = 32
os.environ['OPENBLAS_NUM_THREADS'] = f"{default_n_threads}"
os.environ['MKL_NUM_THREADS'] = f"{default_n_threads}"
os.environ['OMP_NUM_THREADS'] = f"{default_n_threads}"

# import dependencies
import pandas as pd
import numpy as np
import pickle
import pytz
import anndata as ad

from datetime import datetime
from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2
from distributed import Client, LocalCluster

from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.binarization import binarize
from pyscenic.utils import modules_from_adjacencies
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

# Parameters

In [3]:
# [Input] H5ad file to use
EXPRESSION_H5AD_FNAME = '/data/gardeux/Neuro_Droso_ND75KD/data/Kellis_2024/Gene Expression (snRNAseq - 10x) processed/PFC427_raw_data.h5ad' # From Synapse
#EXPRESSION_H5AD_FNAME = '/data/gardeux/Neuro_Droso_ND75KD/data/Kellis_2024/Gene Expression (snRNAseq - 10x) processed, multi-region/all_brain_regions_filt_preprocessed_scanpy_fullmatrix.h5ad' # From Synapse

# [Input] Load expression matrix from H5ad file
f_h5ad = ad.read_h5ad(EXPRESSION_H5AD_FNAME)
f_gene_names = f_h5ad.var_names.tolist()  # Gene names
f_cell_names = f_h5ad.obs_names.tolist()   # Cell names
ex_matrix = pd.DataFrame.sparse.from_spmatrix(f_h5ad.X.T, index=f_gene_names, columns=f_cell_names)

# [Input] Transcription factors list (SCENIC step 1: GRNBoost2)
f_tfs = "/data/gardeux/Neuro_Droso_ND75KD/data/allTFs_hg38.txt" # From https://resources.aertslab.org/cistarget/tf_lists/
# Derive list of Transcription Factors(TF)
tf_names = load_tf_names(f_tfs)

# [Output] Adjacency matrix (SCENIC step 1: GRNBoost2)
adj_matrix = "/data/gardeux/Neuro_Droso_ND75KD/data/Kellis_2024/Kellis_cell.type_adj.csv"

# [Input] Ranking databases (SCENIC step 2-3: cisTarget)
f_db_names = ["/data/gardeux/Neuro_Droso_ND75KD/data/hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather"] # From pySCENIC db: https://resources.aertslab.org/cistarget/databases/homo_sapiens/hg38/refseq_r80/mc_v10_clust/gene_based/
# Alternatively: f_db_names = ["/data/gardeux/Neuro_Droso_ND75KD/data/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather"]
dbs = [RankingDatabase(fname=f_name, name=os.path.basename(f_name)) for f_name in f_db_names]

# [Input] Motif databases (SCENIC step 2-3: cisTarget)
f_motif_path = "/data/gardeux/Neuro_Droso_ND75KD/data/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl" # From pySCENIC db: https://resources.aertslab.org/cistarget/motif2tf/

# [Output] Regulons (SCENIC step 2-3: cisTarget)
f_motifs_path = "/data/gardeux/Neuro_Droso_ND75KD/data/Kellis_2024/Kellis_cell.type_motifs.tsv"
f_modules_path = "/data/gardeux/Neuro_Droso_ND75KD/data/Kellis_2024/Kellis_cell.type_modules.tsv"
f_regulons_path = "/data/gardeux/Neuro_Droso_ND75KD/data/Kellis_2024/Kellis_cell.type_regulons.tsv"
f_regulons_aucell_path = "/data/gardeux/Neuro_Droso_ND75KD/data/Kellis_2024/Kellis_cell.type_regulons_aucell.tsv"
f_regulons_binarized_aucell_path = "/data/gardeux/Neuro_Droso_ND75KD/data/Kellis_2024/Kellis_cell.type_regulons_aucell_binarized.tsv"
f_regulons_binarization_thresholds_aucell_path = "/data/gardeux/Neuro_Droso_ND75KD/data/Kellis_2024/Kellis_cell.type_regulons_aucell_binarization_thresholds.tsv"

# Restrict matrix to feather genes
ranking_feather = pd.read_feather(f_db_names[0])
overlap_values = ex_matrix.index[pd.Series(ex_matrix.index).isin(ranking_feather.columns)].unique()
ex_matrix = ex_matrix.loc[overlap_values, :] # This step takes forever

ex_matrix # 18587 (out of XXX) genes x 1612073 cells for region data  # 20653 (out of 33538) genes x 2663736 cells for cell_type data

,AAACGGGAGATCCCGC-1-0,AAAGATGCACGGTGTC-1-0,AAATGCCTCCAATGGT-1-0,AACCATGTCAGTGCAT-1-0,AACCATGTCTGTACGA-1-0,AACCGCGTCCGCATAA-1-0,AACGTTGGTTCAGGCC-1-0,AACTCCCGTCAATACC-1-0,AACTGGTAGCTAGGCA-1-0,AACTGGTGTACCGGCT-1-0,...,TTTGGAGTCTGGGCGT-16-14,TTTGGTTCAAGGCCTC-16-14,TTTGGTTCACGACAAG-16-14,TTTGGTTCACTCCACT-16-14,TTTGGTTTCGCACTCT-16-14,TTTGTTGAGCCTCCAG-16-14,TTTGTTGCAATACCTG-16-14,TTTGTTGCAGGACATG-16-14,TTTGTTGGTGTGCTTA-16-14,TTTGTTGTCTGCTAGA-16-14
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM87B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DIP2A,0.0,0.0,0.0,0.0,2.0,2.0,3.0,0.0,2.0,1.0,...,0.0,5.0,2.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0
S100B,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0
PRMT2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,4.0,1.0,1.0,5.0,2.0,1.0,4.0,1.0,0.0
MAFIP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Note:** Here the dataset is too big and makes the server to crash (out of RAM!). I will run the pipeline only on excitatory neurons.

In [4]:
# Write h5ad metadata to tsv file
f_h5ad.obs.to_csv("/data/gardeux/Neuro_Droso_ND75KD/data/Kellis_2024/Gene Expression (snRNAseq - 10x) processed/PFC427_raw_data.metadata.tsv", sep = "\t", header=True, index=True)

# Write h5ad metadata to tsv file
f_h5ad.var.to_csv("/data/gardeux/Neuro_Droso_ND75KD/data/Kellis_2024/Gene Expression (snRNAseq - 10x) processed/PFC427_raw_data.gene_metadata.tsv", sep = "\t", header=True, index=True)

In [5]:
# Safety check
(f_h5ad.obs.index == ex_matrix.columns).all()

True

In [6]:
# Subset to only Excitatory neurons
ex_matrix = ex_matrix.loc[:, f_h5ad.obs["major_cell_type"] == "Exc"]
ex_matrix # 20653 rows × 1043230 columns

,AAACGGGAGATCCCGC-1-0,AAATGCCTCCAATGGT-1-0,AACCATGTCAGTGCAT-1-0,AACCATGTCTGTACGA-1-0,AACCGCGTCCGCATAA-1-0,AACGTTGGTTCAGGCC-1-0,AACTGGTGTACCGGCT-1-0,AAGACCTAGTTAACGA-1-0,AAGGAGCAGCAATCTC-1-0,AAGGAGCTCTGCTGCT-1-0,...,TTTAGTCCAGCGTTTA-16-14,TTTCACATCCCTTTGG-16-14,TTTCACATCCGCAGTG-16-14,TTTCCTCAGCGGGTAT-16-14,TTTCGATAGATGACAT-16-14,TTTGGTTCAAGGCCTC-16-14,TTTGGTTCACGACAAG-16-14,TTTGGTTCACTCCACT-16-14,TTTGGTTTCGCACTCT-16-14,TTTGTTGAGCCTCCAG-16-14
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM87B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DIP2A,0.0,0.0,0.0,2.0,2.0,3.0,1.0,0.0,1.0,0.0,...,4.0,3.0,0.0,2.0,0.0,5.0,2.0,0.0,1.0,2.0
S100B,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
PRMT2,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,2.0,0.0,0.0,3.0,1.0,4.0,1.0,1.0,5.0,2.0
MAFIP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# SCENIC steps

## STEP 1: Gene regulatory network inference, and generation of co-expression modules

### 1.a. GRN inference using the GRNBoost2 algorithm

In the initial phase of the pySCENIC pipeline the single cell expression profiles are used to infer co-expression modules from.

Run GRNboost from arboreto to infer co-expression modules

The arboreto package is used for this phase of the pipeline.

*Output:* List of adjacencies between a TF and its targets.

Run GRNBoost2 algorithm

In [ ]:
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

# Prepare the multithreading
cluster = LocalCluster(name='grn_call', dashboard_address=":12345", n_workers=default_n_threads, threads_per_worker=8)
client = Client(cluster)

# Here I run the function within the package (no CLI)
adjacencies = grnboost2(expression_data=ex_matrix.transpose(), tf_names=tf_names, seed=42, verbose=True, client_or_address=client)
            
# Shutting down cluster
client.close()
cluster.close()
    
end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))

# Note: Takes ~18hrs52mn with n_workers=32, threads_per_worker=8

Start time: 18:35:24
preparing dask client
parsing input
creating dask graph


Read in the adjacencies matrix

In [ ]:
adjacencies.to_csv(adj_matrix, index=False, sep=',')
#adjacencies = pd.read_csv(adj_matrix, sep=',', na_filter=False) # If na_filter=True, the nan gene is detected as NaN
adjacencies

## STEP 2-3: Regulon prediction aka cisTarget

*Output:* List of adjacencies between a TF and its targets.

### 2.a. Running regulon prediction using cisTarget

Here, we use the --mask_dropouts option, which affects how the correlation between TF and target genes is calculated during module creation. It is important to note that prior to pySCENIC v0.9.18, the default behavior was to mask dropouts, while in v0.9.18 and later, the correlation is performed using the entire set of cells (including those with zero expression). When using the modules_from_adjacencies function directly in python instead of via the command line, the rho_mask_dropouts option can be used to control this.

**Note:** I kept same parameters than when I ran pySCENIC on our own dataset. It produces 1618 regulons when using TF list from Aerts.

In [ ]:
modules = list(modules_from_adjacencies(adjacencies, ex_matrix.transpose(), rho_mask_dropouts=True, keep_only_activating=True)) # rho_mask_dropouts=True

In [ ]:
modules_df = pd.DataFrame(index = range(0, len(modules)), columns = ("Regulon", "TF", "TFTargetGenesCorrelation", "NbMarkers", "Context", "NES", "Markers"))
for j in range(0, len(modules)):
    # Setting values
    context = list(modules[j].context)
    modules_df["Regulon"].iloc[j] = modules[j].name
    modules_df["TF"].iloc[j] = modules[j].transcription_factor
    modules_df["TFTargetGenesCorrelation"].iloc[j] = context[0]
    modules_df["NbMarkers"].iloc[j] = len(set(modules[j].gene2weight))
    modules_df["Context"].iloc[j] = context[1]
    modules_df["NES"].iloc[j] = modules[j].score
    modules_df["Markers"].iloc[j] = ','.join(list(modules[j].gene2weight))

modules_df = modules_df.sort_values(by='NbMarkers', ascending=False)
modules_df.to_csv(f_modules_path, index=False, sep = "\t")
modules_df

In [ ]:
print(modules_df.TF.nunique(), "unique TF-modules were found ( out of",len(tf_names),"). Modules with less than 20 markers were filtered out.")

In [ ]:
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

df = prune2df(dbs, modules, f_motif_path, num_workers=default_n_threads, weighted_recovery=False, rank_threshold = 1500, nes_threshold=3, motif_similarity_fdr=0.001, auc_threshold=0.05, filter_for_annotation=False)
    
end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))
# Note: 36mn27 with num_workers=12

df.to_csv(f_motifs_path,sep = "\t")
#df

In [ ]:
print(len(set(df.index.get_level_values('TF').values)), "regulons were kept, after pruning")

In [ ]:
# Look for main regulons
print("ATF4", "ATF4" in df.index.get_level_values('TF').values, sep="\t")

In [ ]:
print("Size of Dataframe:", len(df))
drop_indexes = []
for j in range(0, len(df)):
    # Setting values
    if(len(df["Enrichment"]["TargetGenes"][j]) == 0): drop_indexes.append(df.index[j])
df_filtered = df["Enrichment"].drop(index=drop_indexes)
print("Size of Dataframe:", len(df_filtered))

These "modules" are then combined into regulons, by taking the top NES for each TF (for main Motif, and final score of regulon). All genes are bundled together.

In [ ]:
# This dataframe can then be converted to regulons.
regulons = df2regulons(df_filtered)

In [ ]:
regulon_df = pd.DataFrame(index = range(0, len(regulons)), columns = ("Regulon", "TF", "TFTargetGenesCorrelation", "NbMarkers", "Motif", "NES", "Markers"))
for j in range(0, len(regulons)):
    # Fixing order of set
    context = list(regulons[j].context)
    if(context[0].endswith(".png")):
        tmp = context[0]
        context[0] = context[1]
        context[1] = tmp
    # Setting values
    regulon_df["Regulon"].iloc[j] = regulons[j].name
    regulon_df["TF"].iloc[j] = regulons[j].transcription_factor
    regulon_df["TFTargetGenesCorrelation"].iloc[j] = context[0]
    regulon_df["NbMarkers"].iloc[j] = len(set(regulons[j].gene2weight))
    regulon_df["Motif"].iloc[j] = "https://resources.aertslab.org/cistarget/motif_collections/v10nr_clust_public/logos/" + context[1]
    regulon_df["NES"].iloc[j] = regulons[j].score
    regulon_df["Markers"].iloc[j] = ','.join(list(regulons[j].gene2weight))

regulon_df = regulon_df.sort_values(by='NbMarkers', ascending=False)
regulon_df.to_csv(f_regulons_path, index=False, sep = "\t")
regulon_df

In [ ]:
# Look for main regulons
print("ATF4", "ATF4" in df.index.get_level_values('TF').values, sep="\t")

## Phase III: Cellular regulon enrichment matrix (aka AUCell)

Characterize the different cells in a single-cell transcriptomics experiment by the enrichment of the regulons. Enrichment of a regulon is measures as AUC of the recovery curve of the genes that define this regulon.

In [ ]:
auc_mtx = aucell(ex_matrix.transpose(), regulons, num_workers=default_n_threads)
auc_mtx.to_csv(f_regulons_aucell_path, sep = "\t")
auc_mtx

In [ ]:
# Checkpoint to regenerate the object from the file
#auc_mtx = pd.read_csv(f_regulons_aucell_path, sep = "\t", index_col = "Cell")
#auc_mtx.columns.name = "Regulon"
#auc_mtx

In [ ]:
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

auc_mtx_bin = binarize(auc_mtx, seed = 42, num_workers=default_n_threads)

end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))

# Note: 04h21mn56 with num_workers=12

In [ ]:
binarization_thresholds = auc_mtx_bin[1]
binarization_thresholds.to_csv(f_regulons_binarization_thresholds_aucell_path, sep = "\t")
binarization_thresholds

In [ ]:
auc_mtx_bin = auc_mtx_bin[0]
auc_mtx_bin.to_csv(f_regulons_binarized_aucell_path, sep = "\t")
auc_mtx_bin

In [ ]:
sum(auc_mtx_bin["ATF4(+)"])

In [ ]:
binarization_thresholds.loc['ATF4(+)']

In [ ]:
sum(auc_mtx["ATF4(+)"] > binarization_thresholds.loc['ATF4(+)'])